## 🍠  [Day 18](https://adventofcode.com/2018/day/18)

In [0]:
import numpy as np
from scipy.signal import convolve

def display_area(widget, area, step=0):
    """Display the acres area in a HTML Ipython Widget"""
    html_content = ('<center>Step %04d<br><table>' % step + 
                    '\n'.join('<tr>%s</tr>' % (
        ''.join('<td>%s</td>' % (
                    '.' if c == 0 else '|' if c == 1 else '#') 
                for c in line)) 
                             for line in area) + "</table></center>")
    widget.value = html_content
    
    
def one_generation(grid):
    """Apply one generation of the 'Game of Trees'"""
    kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]], 
                      dtype=np.float32)
    # count number of trees around each cell
    trees = (grid == 1)
    num_trees = convolve(trees, kernel, mode='same')
    # count number of lumberyards around each cell
    lumberyards = (grid == 2)
    num_lumberyards = convolve(lumberyards, kernel, mode='same')
    # update rules
    grid[~trees & ~lumberyards & (num_trees >= 3)] = 1
    grid[trees & (num_lumberyards >= 3)] = 2
    grid[lumberyards & ~((num_lumberyards >= 1) & 
                         (num_trees >= 1))] = 0
    return grid

  
def get_resource_value(inputs,
                       num_minutes=10,
                       verbose=False,
                       sleep_delay=0.1):
    """Compute the total resource value after the given number of 
        minutes and display the area's evoluation if `verbose`"""
    grid = np.copy(inputs)
    widget = None
    if verbose:
        widget = ipywidgets.HTML()
        display_area(widget, grid, step=0)
        display(widget)
    for n in range(num_minutes):
        one_generation(grid)
        if widget is not None:
            display_area(widget, grid, step=n + 1)
            time.sleep(sleep_delay)
    return np.sum(grid == 1) * np.sum(grid == 2)

  
def get_resource_value_with_periodicity(inputs, num_minutes=10):
    """Compute the total resource value after the given number of 
        minutes and tries to retrieve periodic pattern if any"""
    grid = np.copy(inputs)
    history = [inputs]
    for n in range(num_minutes):
        one_generation(grid)
        # Test if periodic pattern
        for t, h in enumerate(history):
            if np.all(grid == h):
                print('Periodicity found: [%d, %d]' % (t, n))
                history = history[t:]
                last_frame = (num_minutes - n - 1) % (n + 1 - t)
                grid = history[last_frame]
                return np.sum(grid == 1) * np.sum(grid == 2)
        # Otherwise append to history
        history.append(np.copy(grid))
    return np.sum(grid == 1) * np.sum(grid == 2)

In [4]:
with open('day18.txt', 'r') as f:
  inputs = np.array([
          [0 if char == '.' else 1 if char == '|' else 2
           for char in line] 
            for line in f.read().splitlines() if line.strip()])
    
print('Total Resource value after 10 minutes:', 
      get_resource_value(inputs, num_minutes=10, verbose=False))
print()
print('Total Resource value after... a while:', 
      get_resource_value_with_periodicity(inputs, 
                                          num_minutes=1000000000))

Total Resource value after 10 minutes: 519552

Periodicity found: [568, 595]
Total Resource value after... a while: 165376


In [6]:
## Display example on the test inputs 
# Works in normal ipython
# ipywidgets are not supported with colab so far
import time
import ipywidgets
from IPython.display import display

test_inputs = """.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|."""

test_inputs = np.array([
    [0 if char == '.' else 1 if char == '|' else 2 for char in line] 
    for line in test_inputs.splitlines() if line.strip()])
_ = get_resource_value(test_inputs, num_minutes=18, verbose=True)

.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
